<a href="https://colab.research.google.com/github/jeffheaton/app_generative_ai/blob/main/t81_559_class_05_2_parsers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# T81-559: Applications of Generative Artificial Intelligence
**Module 5: LangChain: Data Extraction**
* Instructor: [Jeff Heaton](https://sites.wustl.edu/jeffheaton/), McKelvey School of Engineering, [Washington University in St. Louis](https://engineering.wustl.edu/Programs/Pages/default.aspx)
* For more information visit the [class website](https://sites.wustl.edu/jeffheaton/t81-558/).

# Module 5 Material

* Part 5.1: Structured Output Parser [[Video]](https://www.youtube.com/watch?v=62CSR141VRE) [[Notebook]](t81_559_class_05_1_langchain_data.ipynb)
* **Part 5.2: Other Parsers (CSV, JSON, Pandas, Datetime)** [[Video]](https://www.youtube.com/watch?v=VXm8gPzU3qc) [[Notebook]](t81_559_class_05_2_parsers.ipynb)
* Part 5.3: Pydantic parser [[Video]](https://www.youtube.com/watch?v=dc4fn-W60hg) [[Notebook]](t81_559_class_05_3_pydantic.ipynb)
* Part 5.4: Custom Output Parser [[Video]](https://www.youtube.com/watch?v=jBpkAblQC_U) [[Notebook]](t81_559_class_05_4_custom_parsers.ipynb)
* Part 5.5: Output-Fixing Parser [[Video]](https://www.youtube.com/watch?v=_txWiLjf4bo) [[Notebook]](t81_559_class_05_5_output_fixing_parsers.ipynb)

# Google CoLab Instructions

The following code ensures that Google CoLab is running and maps Google Drive if needed.

In [9]:
import os

try:
    from google.colab import drive, userdata
    COLAB = True
    print("Note: using Google CoLab")
except:
    print("Note: not using Google CoLab")
    COLAB = False

# OpenAI Secrets
if COLAB:
    os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

# Install needed libraries in CoLab
if COLAB:
    !pip install langchain langchain_openai

Note: using Google CoLab


# 5.2: Other Parsers (Comma List, JSON, Pandas, Datetime)


In this section, we'll explore how LangChain offers versatile parsers capable of handling a variety of data formats, enhancing its functionality across numerous applications. Among these, it can seamlessly integrate with data in the form of Pandas dataframes, comma-separated lists, JSON structures, and datetime objects, among others. This capability ensures that LangChain can adapt to diverse data inputs, making it a powerful tool for data manipulation and analysis in different contexts. We will delve into some of these parsers and demonstrate their practical applications, highlighting how they can be utilized to streamline processes and extract meaningful insights from data.

## Parse Comma Separated List Response

We will begin with the CommaSeparatedListOutputParser parser, which can take the LLM output in a comma-separated list and extract it as a Python list.

In [10]:
from langchain.output_parsers import CommaSeparatedListOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

output_parser = CommaSeparatedListOutputParser()

format_instructions = output_parser.get_format_instructions()
prompt = PromptTemplate(
    template="List ten {subject}.\n{format_instructions}",
    input_variables=["subject"],
    partial_variables={"format_instructions": format_instructions},
)

MODEL = "gpt-5-mini"
TEMPERATURE = 0

# Initialize the OpenAI LLM with your API key
llm = ChatOpenAI(
    model=MODEL,
    temperature=TEMPERATURE,
    n=1
)

chain = prompt | llm | output_parser

Extract a list of cities.

In [11]:
chain.invoke({"subject": "cities"})

['New York',
 'London',
 'Tokyo',
 'Paris',
 'Sydney',
 'Cairo',
 'Mumbai',
 'São Paulo',
 'Toronto',
 'Berlin']

Extract a list of programming languages.

In [12]:
chain.invoke({"subject": "programming languages"})

['Python',
 'Java',
 'C',
 'C++',
 'JavaScript',
 'Ruby',
 'Go',
 'Swift',
 'Rust',
 'PHP']

## Parse JSON Response

We can format the output from the LLM into JSON. For this example, we will accept a sentence that we detect as English and then translate it into Spanish, French, and Chinese.

In [13]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import ChatOpenAI

# Define your desired data structure.
class Translate(BaseModel):
  detected: str = Field(description="the detected language of the input")
  spanish: str = Field(description="the input translated to Spanish")
  french: str = Field(description="the input translated to French")
  chinese: str = Field(description="the input translated to Chinese")

# And a query intented to prompt a language model to populate the data structure.
input_text = "What is your name?"

# Set up a parser + inject instructions into the prompt template.
parser = JsonOutputParser(pydantic_object=Translate)

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{input}\n",
    input_variables=["input"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

chain = prompt | llm | parser

chain.invoke({"input": input_text})

{'detected': 'English',
 'spanish': '¿Cómo te llamas?',
 'french': 'Comment vous appelez-vous ?',
 'chinese': '你叫什么名字？'}

## Query Pandas Dataframe

Langchain's capabilities include parsing and analyzing Pandas dataframes using the PandasDataFrameOutputParser. This feature allows users to seamlessly integrate data stored in Pandas dataframes and use Langchain to query and extract insights from this data. By leveraging the PandasDataFrameOutputParser, Langchain can interpret the dataframe's structure, contents, and context, enabling it to provide accurate answers to user queries. This integration is particularly useful for data analysis, enabling more interactive and natural language-based exploration of data stored in Pandas dataframes.

The following code reads and displays the first lines from the classic [iris dataset](https://scikit-learn.org/stable/auto_examples/datasets/plot_iris_dataset.html).

In [14]:
import pprint
from typing import Any, Dict

import pandas as pd
from langchain.output_parsers import PandasDataFrameOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# Load the iris dataset
df = pd.read_csv(
    "https://data.heatonresearch.com/data/t81-558/iris.csv", na_values=["NA", "?"]
)

print(df.head())

   sepal_l  sepal_w  petal_l  petal_w      species
0      5.1      3.5      1.4      0.2  Iris-setosa
1      4.9      3.0      1.4      0.2  Iris-setosa
2      4.7      3.2      1.3      0.2  Iris-setosa
3      4.6      3.1      1.5      0.2  Iris-setosa
4      5.0      3.6      1.4      0.2  Iris-setosa


Next we load the iris dataframe into a PandasDataFrameOutputParser class.

In [15]:
parser = PandasDataFrameOutputParser(dataframe=df)

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

chain = prompt | llm | parser

We query for the mean of one of the columns.

In [16]:
query = "Get the mean of the sepal_l column."
parser_output = chain.invoke({"query": query})
print(parser_output)

{'mean': np.float64(5.843333333333334)}


We query for the sub of one of the columns.

In [17]:
query = "Get the sum of petal_w column."
parser_output = chain.invoke({"query": query})
print(parser_output)

{'sum': np.float64(179.90000000000003)}


## Datetime

Langchain includes a feature known as the DatetimeOutputParser, which is specifically designed to parse datetime values from text. This capability allows it to recognize and interpret dates and times expressed in various formats, converting them into a standardized datetime format. This functionality is invaluable in applications involving scheduling, data analysis, or any context where accurate handling of dates and times is essential. By utilizing the DatetimeOutputParser, developers can streamline the processing of temporal data, ensuring that their applications can effectively manage and respond to time-related information.

In [18]:
from langchain.output_parsers import DatetimeOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import OpenAI

output_parser = DatetimeOutputParser()
template = """Answer the users question:

{question}

{format_instructions}"""
prompt = PromptTemplate.from_template(
    template,
    partial_variables={"format_instructions": output_parser.get_format_instructions()},
)

We can display that prompt that we will use to obtain dates.

In [19]:
print(prompt)

input_variables=['question'] input_types={} partial_variables={'format_instructions': "Write a datetime string that matches the following pattern: '%Y-%m-%dT%H:%M:%S.%fZ'.\n\nExamples: 2023-07-04T14:30:00.000000Z, 1999-12-31T23:59:59.999999Z, 2025-01-01T00:00:00.000000Z\n\nReturn ONLY this string, no other words!"} template='Answer the users question:\n\n{question}\n\n{format_instructions}'


We create the chain that we will use to parse dates.

In [20]:
chain = prompt | llm | output_parser

We will query for two dates, one real and the other fictional.

In [21]:
output = chain.invoke({"question": "When was the Python language introduced?"})
print(output)

1991-02-20 00:00:00


In [22]:
output = chain.invoke({"question": "What is the date of the war in the video game Fallout?"})
print(output)

2077-10-23 00:00:00
